In [1]:
import pandas as pd
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import collections
from gensim.models import doc2vec
from sklearn.model_selection import train_test_split
import os
import shutil
import datetime
# from tensorboardX import SummaryWriter
# from gluoncv.utils import TrainingHistory
import warnings
import numpy as np

In [2]:
import os

import tensorflow as tf
from keras import layers
# from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras import backend as K
from keras.models import Model
import keras
from keras_self_attention import SeqSelfAttention
from keras.layers import Lambda,Conv1D, MaxPooling1D,Input, Flatten, Dense,Reshape,Concatenate,LSTM,GlobalAveragePooling1D
from keras.backend import slice
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
label = pd.read_csv('D:/paper/paper0311/binary_label.csv')
label['label'] = label['label'].astype(str)
label['label'].replace('10','010',inplace = True)
label['label'].replace('1','001',inplace = True)
label['label'].replace('11','011',inplace = True)

# 단일질병 당뇨, 고혈압, 동맥경화 각 10명씩 추출하기

In [47]:
single_d =[]
single_h =[]
single_b =[]

for i in label[label['label']=='100'].PERSON_ID:
    single_d.append(i)
for i in label[label['label']=='010'].PERSON_ID:
    single_h.append(i)
for i in label[label['label']=='001'].PERSON_ID:
    single_b.append(i)

In [48]:
# 각 질병 별로 10명의 ID 추출
import random
from itertools import combinations
single_diabet = []
single_hyper = []
single_blood = []
random.seed(1)
count=10
for i in range(10):
    d = random.choice(single_d)
    single_diabet.append(d)
    h = random.choice(single_h)
    single_hyper.append(h)
    b = random.choice(single_b)
    single_blood.append(b)

In [49]:
import tensorflow.keras.backend as K
import tensorflow as tf
def contrastive_loss(y, preds, margin=1):
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

    return loss

In [50]:
from keras.models import load_model
single_model = load_model('D:/paper/0607/single_similarity.h5',custom_objects={'SeqSelfAttention':SeqSelfAttention,
                                                                                     'contrastive_loss':contrastive_loss})

In [18]:
embedding=np.load('D:/paper/paper0311/embedding_metrix.npy')
embedding.shape

(142073, 200, 128)

In [51]:
data=pd.read_csv('D:/paper/paper0311/final_prepro0311.csv')
mask = data['PERSON_ID'].value_counts()[(data['PERSON_ID'].value_counts()>=4)].index
alldata = data[data.PERSON_ID.isin(mask)].reset_index(drop=True)
person_id=alldata['PERSON_ID'].unique()
alldata.sort_values('RECU_FR_DT',inplace=True)
alldata.drop(columns=['KEY_SEQ','RECU_FR_DT','MAIN_ICDnorm'],inplace=True)
person_id=person_id.tolist()
# test_label_m=pd.read_csv('D:/paper/0607/test_m.csv')
# test_label_s=pd.read_csv('D:/paper/0607/test_s.csv')
person_doc_dict={key:value for value,key in enumerate(person_id)}

In [52]:
group1_index = [person_doc_dict[person_id] for person_id in single_diabet]
group2_index = [person_doc_dict[person_id] for person_id in single_hyper]
group3_index = [person_doc_dict[person_id] for person_id in single_blood]

In [53]:
group1_emb = embedding[group1_index, :]
group1_emb = np.array(group1_emb)

group2_emb = embedding[group2_index, :]
group2_emb = np.array(group2_emb)

group3_emb = embedding[group3_index, :]
group3_emb = np.array(group3_emb)

In [54]:
def get_single_pair():
    index_list =[i for i in range(10)]
    pairs = list(combinations(index_list, 2))
    batch_1 = [i for i,j in pairs]
    batch_2=[j for i,j in pairs]
    return batch_1, batch_2
batch_1, batch_2= get_single_pair()

In [55]:
def get_double_pair():
    batch_list=[(i,j) for i in range(10) for j in range(10)]
    print('pairs count:', len(batch_list))
    batch1 = [i for i,j in batch_list]
    batch2 = [j for i,j in batch_list]
    return batch1, batch2
batch_group1, batch_group2 = get_double_pair()

pairs count: 100


In [160]:
def result(arr):
    print('mean:', arr.mean())
    print('max:', arr.max())
    print('min:', arr.min())
    print('-------------')

In [68]:
# 당뇨&당뇨/ 고혈압&고혈압/ 동맥&동맥
result_d = single_model.predict([group1_emb[batch_1],group1_emb[batch_2]])
result_h = single_model.predict([group2_emb[batch_1],group2_emb[batch_2]])
result_b = single_model.predict([group3_emb[batch_1],group3_emb[batch_2]])


In [161]:
result(1-result_d[0])
result(1-result_h[1])
result(1-result_b[2])

mean: 0.97770005
max: 0.9959625
min: 0.86490524
-------------
mean: 0.9950759
max: 0.99844897
min: 0.9914564
-------------
mean: 0.37746242
max: 0.9999406
min: 0.0
-------------


In [162]:
diabet_s =(3 - (result_d[0]+result_d[1]+result_d[2]))/3
hyper_s =(3 - (result_h[0]+result_h[1]+result_h[2]))/3
blood_s =(3 - (result_b[0]+result_b[1]+result_b[2]))/3

result(diabet_s)
result(hyper_s)
result(blood_s)

mean: 0.98429954
max: 0.99402386
min: 0.9241588
-------------
mean: 0.99268806
max: 0.9978821
min: 0.98902553
-------------
mean: 0.6339434
max: 0.9949537
min: 0.30938068
-------------


In [163]:
# 당뇨&고/ 고&당/ 당&동맥
result_d_h = single_model.predict([group1_emb[batch_group1],group2_emb[batch_group2]])
result_d_b = single_model.predict([group1_emb[batch_group1],group3_emb[batch_group2]])
result_h_b = single_model.predict([group2_emb[batch_group1],group3_emb[batch_group2]])

In [164]:
b1 =3-(result_d_h[0]+result_d_h[1]+result_d_h[2])
b2 =3-(result_d_b[0]+result_d_b[1]+result_d_b[2])
b3 =3-(result_h_b[0]+result_h_b[1]+result_h_b[2])


result(b1/3)
result(b2/3)
result(b3/3)

mean: 0.34235698
max: 0.39635137
min: 0.3303453
-------------
mean: 0.09164371
max: 0.5070984
min: 0.0024122398
-------------
mean: 0.39122975
max: 0.9902477
min: 0.31020555
-------------


# 다중 질병 분석

In [165]:
multi_d_h =[]
multi_d_b =[]
multi_h_b =[]
multi_d_h_b =[]

for i in label[label['label']=='110'].PERSON_ID:
    multi_d_h.append(i)
for i in label[label['label']=='101'].PERSON_ID:
    multi_d_b.append(i)
for i in label[label['label']=='011'].PERSON_ID:
    multi_h_b.append(i)
for i in label[label['label']=='111'].PERSON_ID:
    multi_d_h_b.append(i)

In [166]:
# 다중질병 환자 별로 10명의 ID 추출
d_h = []
d_b = []
d_h_b =[]
h_b = []
count = 10
random.seed(1)
for i in range(count):
    d_h.append(random.choice(multi_d_h))
    d_b.append(random.choice(multi_d_b))
    h_b.append(random.choice(multi_h_b))
    d_h_b.append(random.choice(multi_d_h_b))

In [184]:
joint_model = load_model('D:/paper/0607/proposed_attention_model0426.h5',custom_objects={'SeqSelfAttention':SeqSelfAttention,
                                                                                     'contrastive_loss':contrastive_loss})
multi_model = load_model('D:/paper/0607/multi_similarity.h5',custom_objects={'SeqSelfAttention':SeqSelfAttention,
                                                                                     'contrastive_loss':contrastive_loss})

In [189]:
g1_index = [person_doc_dict[person_id] for person_id in d_h]
g2_index = [person_doc_dict[person_id] for person_id in d_b]
g3_index = [person_doc_dict[person_id] for person_id in h_b]
g4_index = [person_doc_dict[person_id] for person_id in d_h_b]

In [190]:
g1_emb = embedding[g1_index, :]
g1_emb = np.array(g1_emb)

g2_emb = embedding[g2_index, :]
g2_emb = np.array(g2_emb)

g3_emb = embedding[g3_index, :]
g3_emb = np.array(g3_emb)

g4_emb = embedding[g4_index, :]
g4_emb = np.array(g4_emb)

In [202]:
result_d_h = joint_model.predict([g1_emb[batch_1],g1_emb[batch_2]])
result_d_b = joint_model.predict([g2_emb[batch_1],g2_emb[batch_2]])
result_h_b = joint_model.predict([g3_emb[batch_1],g3_emb[batch_2]])
result_d_b_h = joint_model.predict([g4_emb[batch_1],g4_emb[batch_2]])

In [204]:
c1 = 3-(result_d_h[0]+result_d_h[1]+result_d_h[2])
c2 = 3-(result_d_b[0]+result_d_b[1]+result_d_b[2])
c3 = 3-(result_h_b[0]+result_h_b[1]+result_h_b[2])
c4 = 3-(result_d_h_b[0]+result_d_h_b[1]+result_d_h_b[2])

result(c1/3)
result(c2/3)
result(c3/3)
result(c4/3)

mean: 0.8917903
max: 0.986247
min: 0.71320254
-------------
mean: 0.9436907
max: 0.9770744
min: 0.8543894
-------------
mean: 0.967227
max: 0.9828665
min: 0.94553477
-------------
mean: 0.97215956
max: 0.99604625
min: 0.9205981
-------------


In [209]:
a1 = joint_model.predict([g1_emb[batch_group1],g2_emb[batch_group2]])# d_h,d_b
a2 =joint_model.predict([g1_emb[batch_group1],g3_emb[batch_group2]])# d_h, h_b
a3 = joint_model.predict([g1_emb[batch_group1],g4_emb[batch_group2]])#d_h d_h_b

a4 = joint_model.predict([g2_emb[batch_group1],g3_emb[batch_group2]])# d_b h_b
a5 = joint_model.predict([g2_emb[batch_group1],g4_emb[batch_group2]]) # d_b d_h_b

a6 = joint_model.predict([g3_emb[batch_group1],g4_emb[batch_group2]]) # h_b d_h_b

In [210]:
a11 = 3-(a1[0]+a1[1]+a1[2])
a22 = 3-(a2[0]+a2[1]+a2[2])
a33 = 3-(a3[0]+a3[1]+a3[2])
a44 = 3-(a4[0]+a4[1]+a4[2])
a55 = 3-(a5[0]+a5[1]+a5[2])
a66 = 3-(a6[0]+a6[1]+a6[2])

result(a11/3)
result(a22/3)
result(a33/3)
result(a44/3)
result(a55/3)
result(a66/3)

mean: 0.6555903
max: 0.87896675
min: 0.3906949
-------------
mean: 0.5176883
max: 0.86155695
min: 0.38010302
-------------
mean: 0.6749493
max: 0.8277831
min: 0.42371318
-------------
mean: 0.71126264
max: 0.89690024
min: 0.6346264
-------------
mean: 0.9634049
max: 0.9960595
min: 0.79820466
-------------
mean: 0.75930154
max: 0.94734025
min: 0.67392784
-------------


In [217]:
# 3개와 1가지 질병
z1 = joint_model.predict([g4_emb[batch_group1], group1_emb[batch_group2]])
z2= joint_model.predict([g4_emb[batch_group1], group2_emb[batch_group2]])
z3 = joint_model.predict([g4_emb[batch_group1], group3_emb[batch_group2]])

# 2개와 1가지질병 
z4 = joint_model.predict([g1_emb[batch_group1], group1_emb[batch_group2]])
z5 = joint_model.predict([g1_emb[batch_group1], group2_emb[batch_group2]])
z6 = joint_model.predict([g1_emb[batch_group1], group3_emb[batch_group2]])

z7 = joint_model.predict([g2_emb[batch_group1], group1_emb[batch_group2]])
z8 = joint_model.predict([g2_emb[batch_group1], group2_emb[batch_group2]])
z9 = joint_model.predict([g2_emb[batch_group1], group3_emb[batch_group2]])

z10 = joint_model.predict([g3_emb[batch_group1], group1_emb[batch_group2]])
z11 = joint_model.predict([g3_emb[batch_group1], group2_emb[batch_group2]])
z12 = joint_model.predict([g3_emb[batch_group1], group3_emb[batch_group2]])
# 3개와 2가지 질병
z13 = joint_model.predict([g4_emb[batch_group1], g1_emb[batch_group2]])
z14 = joint_model.predict([g4_emb[batch_group1], g2_emb[batch_group2]])
z15 = joint_model.predict([g4_emb[batch_group1], g3_emb[batch_group2]])

In [218]:
x1 = 3-(z1[0]+z1[1]+z1[2])
x2 = 3-(z2[0]+z2[1]+z2[2])
x3 = 3-(z3[0]+z3[1]+z3[2])
x4 = 3-(z4[0]+z4[1]+z4[2])
x5 = 3-(z5[0]+z5[1]+z5[2])
x6 = 3-(z6[0]+z6[1]+z6[2])
x7 = 3-(z7[0]+z7[1]+z7[2])
x8 = 3-(z8[0]+z8[1]+z8[2])
x9 = 3-(z9[0]+z9[1]+z9[2])
x10 = 3-(z10[0]+z10[1]+z10[2])
x11 = 3-(z11[0]+z11[1]+z11[2])
x12 = 3-(z12[0]+z12[1]+z12[2])
x13 = 3-(z13[0]+z13[1]+z13[2])
x14 = 3-(z14[0]+z14[1]+z14[2])
x15 = 3-(z15[0]+z15[1]+z15[2])

In [219]:
result(x1/3)
result(x2/3)
result(x3/3)
result(x4/3)
result(x5/3)
result(x6/3)
result(x7/3)
result(x8/3)
result(x9/3)
result(x10/3)
result(x11/3)
result(x12/3)
result(x13/3)
result(x14/3)
result(x15/3)

mean: 0.6934891
max: 0.8159295
min: 0.55888224
-------------
mean: 0.48033863
max: 0.7176318
min: 0.39348006
-------------
mean: 0.7546826
max: 0.9168641
min: 0.42882618
-------------
mean: 0.9242437
max: 0.9810946
min: 0.72317785
-------------
mean: 0.80298775
max: 0.9746365
min: 0.6909764
-------------
mean: 0.55185294
max: 0.96728903
min: 0.39367214
-------------
mean: 0.6299097
max: 0.8382778
min: 0.45718884
-------------
mean: 0.44899473
max: 0.625972
min: 0.36820722
-------------
mean: 0.67141694
max: 0.865829
min: 0.40312982
-------------
mean: 0.47939733
max: 0.66381156
min: 0.37183633
-------------
mean: 0.70044935
max: 0.82165766
min: 0.6174549
-------------
mean: 0.9283242
max: 0.9819563
min: 0.72453594
-------------
mean: 0.6749493
max: 0.8277831
min: 0.42371318
-------------
mean: 0.96340483
max: 0.9960595
min: 0.79820466
-------------
mean: 0.7593014
max: 0.94734025
min: 0.67392784
-------------


In [ ]:

# 단일 질병 샘플 추출하기 